In [103]:
import pandas as pd
import numpy as np

### Assign taxonomy to deblur sOTUs using QIIME2 (qiime.org)

The file 'rep_sotu.qza' artifact contains representative deblur 16S sequences. We assign taxonomy using sckit-learn classifier and greengenes (v13_8) tree.

In [104]:
! qiime feature-classifier classify-sklearn \
  --i-classifier ../../gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads ./data/rep_sotus.qza \
  --o-classification taxonomy_all.qza

/bin/sh: qiime: command not found


In [105]:
! qiime metadata tabulate \
  --m-input-file ./data/taxonomy_all.qza \
  --o-visualization ./data/taxonomy_all.qzv

/bin/sh: qiime: command not found


In [ ]:
! qiime tools export \
  ./data/taxonomy_all.qza \
  --output-dir ./data/deblur_taxonomy_all

In [90]:
sotu=pd.read_table('./data/haddad_6weeks_deblur_otus_rare2k_matched.txt', sep='\t', dtype=str, index_col=0)
taxa=pd.read_table('./data/deblur_taxonomy_all/taxonomy.tsv', sep='\t', dtype=str, index_col="Feature ID")

In [91]:
sotu.head(2)

,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGACTGTCAAGTCAGCGGTAAAATTGAGAGGCTCAACCTCTTCCCGCCGTTGAAACTGGTGGTCTTGAGTGGATGAGAAGTACGCGGAATGCGTGGT,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGACAGTTAAGTCAGCGGTAAAATTGAGAGGCTCAACCTCTTCCCGCCGTTGAAACTGATTGTCTTGAGTGGGCGAGAAGTATGCGGAATGCGTGGT,TACGTAGGTGGCGAGCGTTATCCGGAATGATTGGGCGTAAAGGGTGCGCAGGCGGTCCTGCAAGTCTGGAGTGAAACGCATGAGCTCAACTCATGCATGGCTTTGGAAACTGGAGGACTGGAGAGCAGGAGAGGGCGGTGGAACTCCATG,TACGTAGGTGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGAGGGAGCAGGCGGCAGCAAAGGTCTGTGGTGAAAGACTGAAGCTTAACTTCAGTAAGCCATAGAAACCGGGCAGCTAGAGTGCAGGAGAGGATCGTGGAATTCCATGT,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGGGCGCAGACGGCGATGCAAGCCAGGAGTGAAAGCCCGGGGCCCAACCCCGGGACTGCTCTTGGAACTGCGTGGCTGGAGTGCAGGAGGGGCAGGCGGAATTCCTGG,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGTCCGTTAAGTCAGCGGTAAAATTGCGGGGCTCAACCCCGTCGAGCCGTTGAAACTGGCAGACTTGAGTTGGCGAGAAGTACGCGGAATGCGCGGT,TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGGCGGCCTGCCAAGTCTGATGTGAAATACCGGGGCTCAACCCCGGGGCTGCATTGGAAACTGGCAGGCTGGAGTGTCGGAGAGGCAGGCGGAATTCCTAG,TACGTAGGTGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGCGTGTAGGCGGGAAAGCAAGTCAGATGTGAAAACCATGGGCTCAACCTGTGGCCTGCATTTGAAACTGTTTTTCTTGAGTACTGGAGAGGCAGACGGAATTCCTAG,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGCAGGCGGACTCTCAAGTCAGCGGTCAAATCGCGGGGCTCAACCCCGTTCCGCCGTTGAAACTGGGAGCCTTGAGTGCGCGAGAAGTAGGCGGAATGCGTGGT,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGTGATGCAAGTCTGAAGTGAAAGGCGGGGGCTCAACCCCCGGACTGCTTTGGAAACTGTATGACTGGAGTGCAGGAGAGGTAAGTGGAATTCCTAG,...,TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTATGTAAGACAGTTGTGAAATCCCCGGGCTCAACCTGGGAACTGCATCTGTGACTGCATAGCTAGAGTACGGTAGAGTGGGATGGAATTCCGCG,TACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGTGTAAAGGGCGTGTAGCCGGGTTGACAAGTCAGATGTGAAATCCTGCGGCTTAACCGCAGAACTGCATTTGAAACTGTTGATCTTGAGTACTGGAGAGGCAGACGGAAATCCTAG,TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGCAGGCGGACTCTCAAGTCAGCGGTCAAATCGCGGGGCTCAACCCCGTTCCGCCGTTGAAACTGGTGGTCTTGAGTGGATGAGAAGTACGCGGAATGCGTGGT,TATAAGACAGATGTGAAATCCCCGGGCTCAACCTGGGACCTGCATTTGTGACTGTATCGCTAGAGTACGGTAGAGGGGGATGGAATTCCGCGTGTAGCAGTGAAATGCGTAGATATGCGGAGGAACACCGATGGCGAAGGCAATCCCCTG,TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTATGTAAGACAGTTGTGAAATCCCCGGGCTCAACCTGGGAACTGCATCTGTGACTGCATAGCTAGAGTACGGTAGAGTGGGATGGAATTCCGCGT,TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCCATGCAAGCCTGGAGTGAAAGCCCGGGGCCCAACCCCGGGACTGCTCTGGGAACTGTGCGGCTGGAGTGCGGGAGGGGCAGGCGGAATTCCTGG,TACGTAGGTGGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGCGTGTAGGCGGGAAAGCAAGTCAGATGTGAAAACCATGGGCTCAACCTGTGGCCTGCATTTGAAACTGTTTTTCTTGAGTACTGGAGAGGCAGACGGACTTCCTAG,TACGAAGGGGTCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGATATTTAAGTCAGAGGTGAAATCCCAGGGCTCAACCCTGGAACTGCCTTTGATACTGGGTATCTAGAGTATGGAAGAGGTGAGTGGAATTCCGAG,TACGTAGGGTGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTTGTAGGCGGTTTGTCGCGTCTGCTGTGAAAGACCGGGGCGTAACTCCGGTTCTGCAGTGGGTACGGGCAGACTAGAGTGTGGTAGGGGAGACTGGAATTCCTGG,TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGAGTGCGCAGGCGGTTTTGCAAGACCGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATTGGTGACTGCAAGGCTAGAGTGTGTCAGAGGGAGGTGGGATTCCGCA
#SampleID,,,,,,,,,,,,,,,,,,,,,
10422.17.F.10,856.0,370.0,0.0,135.0,97.0,75.0,0.0,13.0,40.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10422.17.F.11,725.0,302.0,0.0,51.0,223.0,51.0,0.0,88.0,23.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
taxa.head(2)

,Taxon,Confidence
Feature ID,,
TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGACTGTCAAGTCAGCGGTAAAATTGAGAGGCTCAACCTCTTCCCGCCGTTGAAACTGGTGGTCTTGAGTGGATGAGAAGTACGCGGAATGCGTGGT,k__Bacteria; p__Bacteroidetes; c__Bacteroidia;...,0.9999989963431722
TACGGAGGATGCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGACAGTTAAGTCAGCGGTAAAATTGAGAGGCTCAACCTCTTCCCGCCGTTGAAACTGATTGTCTTGAGTGGGCGAGAAGTATGCGGAATGCGTGGT,k__Bacteria; p__Bacteroidetes; c__Bacteroidia;...,0.9999997142501922


In [93]:
#subset taxonomy dataframe to include just the sotus in our sotu table
taxa=taxa.loc[sotu.columns]
taxa.shape

(735, 2)

In [94]:
taxonomy = [taxa.loc[seq, 'Taxon'] for seq in sotu.columns]
sotu.loc['taxonomy'] = taxonomy

In [98]:
sotu = sotu.T.reset_index().rename(columns = {'index': '#OTU ID'})

In [101]:
sotu.to_csv("./data/haddad_6weeks_deblur_otus_rare2k_matched_wtaxa.txt", sep='\t', index = False)

In [102]:
! biom convert -i ./data/haddad_6weeks_deblur_otus_rare2k_matched_wtaxa.txt \
-o ./data/haddad_6weeks_deblur_otus_rare2k_matched_wtaxa.biom \
--to-hdf5 --table-type="OTU table" --process-obs-metadata taxonomy

### Molecule annotations

#### level 2 identification using MS2-based library matches in GNPS (gnps.ucsd.edu)

In [23]:
gnpsf = pd.read_table('./data/allFeatures_gnpsresult.tsv',
                       sep = '\t', dtype = str)
feature = pd.read_table('./data/haddad_6weeks_allFeatures_pqn_matched.txt', 
                        sep ='\t', dtype = str, index_col = '#SampleID')

In [24]:
feature = feature.T.reset_index().rename(columns = {'index': '#featureID'})

In [25]:
feature.head(2)

#SampleID,#featureID,10422.17.F.10,10422.17.F.11,10422.17.F.12,10422.17.F.13,10422.17.F.3,10422.17.F.4,10422.17.F.5,10422.17.F.6,10422.17.F.7,...,10422.32.F.10,10422.32.F.11,10422.32.F.12,10422.32.F.3,10422.32.F.4,10422.32.F.5,10422.32.F.6,10422.32.F.7,10422.32.F.8,10422.32.F.9
0,132.1020923761221_0.4491537660256411,417818365.0404715,599176692.2172579,271010923.09797156,374372770.02204,394579148.9187016,237869662.99171597,606701783.7713683,148560675.25142795,422735443.6537102,...,529147349.75244075,757521148.2018569,216397057.9716737,627558146.4828238,278835766.3882692,591675934.303529,572151910.6403494,439901099.2879787,304919206.69614714,721050657.3552696
1,166.0862755041856_0.5858793269230769,363477950.08953905,651303189.4392803,223933384.08641553,367612132.7783069,363360496.3342866,223205801.5668584,596553038.5969964,99965642.3202608,363523866.8105213,...,483685509.32293975,954922661.5036495,193515512.73507348,744830302.8263001,276344621.4609494,601801823.8958603,609614808.6123556,478803365.0144199,266983835.86724555,959568044.1300417


In [26]:
#split feature ID to m/z and RT (in secs); add new columns

mz=[float(x.split('_')[0]) for x in feature['#featureID']]
rt=[float(x.split('_')[1])*60 for x in feature['#featureID']] # save RT values in secs
feature['obs_mz']=mz
feature['obs_rt']=rt

In [27]:
#look up gnps file (error tolerance: m/z ppm <10 ppm; Retention time tolerance < 30 sec)
#add new column 'level2_gnps' to include library hit "Compound Name"

feature['level2_gnps']=np.nan
feature['ppm_err']=np.nan
feature['rt_err']=np.nan

for idx1 in feature.index:
    for idx2 in gnpsf.index:
        mactual=float(gnpsf.loc[idx2,'SpecMZ'])
        mobs=float(feature.loc[idx1,'obs_mz'])
        ppme=abs((mactual-mobs)*1000000/float(mactual))
        
        tactual=float(gnpsf.loc[idx2,'RT_Query'])
        tobs=float(feature.loc[idx1,'obs_rt'])
        rte=abs((tactual-tobs))
        
        
        if ppme < 10 and rte < 30:
            if pd.isnull(feature.loc[idx1, 'level2_gnps']): #if feature hasn't been assigned any lib match yet
                feature.loc[idx1, 'level2_gnps'] = gnpsf.loc[idx2,'Compound_Name']
                feature.loc[idx1, 'ppm_err']=ppme
                feature.loc[idx1, 'rt_err']=rte
            #if feature was assigned lib match but new match seems more accurate (lesser mz error & rt error < 30s)
            elif ppme < feature.loc[idx1, 'ppm_err'] and rte < 30:
                feature.loc[idx1, 'level2_gnps'] = gnpsf.loc[idx2,'Compound_Name']
            # if feature was assigned lib match which is more accurate than the new match, make no change
            else: 
                continue

In [34]:
# fill NANs woth "none"
feature['level2_gnps']=feature['level2_gnps'].fillna(value="none")

#delete 'ppm_err' and 'rt_err' columns for gnps annotations
feature.drop(['ppm_err', 'rt_err'], axis=1, inplace=True)

#### level 1 annotation for bile acids using comparision to analytical standard 

In [62]:
ids = pd.read_table("./data/bile_acids_standard_identification.txt", sep = '\t', 
                     dtype = str)

ids = ids.drop('Unnamed: 9', axis =1)
ids.fillna(-1, inplace = True)
ids.head(4)

,Name,Abbreviation,Mol Formula,Theoretical m/z,-1 H2O,-2 H2O,-3 H2O,Dimer,Trimer,RT standard QE,RT sec
0,Cholic acid,CA,C24H40O5,409.295,391.285,373.275,355.265,817.582,1225.89,4.23,253.8
1,Deoxycholic acid,DCA,C24H40O4,393.3,375.29,357.28,339.268,785.593,1177.91,4.91,294.6
2,Glycocholic acid,GCA,C26H43NO6,466.317,448.307,430.297,412.287,931.623,1396.96,3.8,228
3,Glycochenodeoxycholic acid,GCDCA,C26H43NO5,450.322,432.312,414.302,-1,899.635,1348.97,4.31,258.6


In [63]:
feature.head(2)

#SampleID,#featureID,10422.17.F.10,10422.17.F.11,10422.17.F.12,10422.17.F.13,10422.17.F.3,10422.17.F.4,10422.17.F.5,10422.17.F.6,10422.17.F.7,...,10422.32.F.6,10422.32.F.7,10422.32.F.8,10422.32.F.9,obs_mz,obs_rt,level2_gnps,level_1_identification,ppm_err,rt_err
0,132.1020923761221_0.4491537660256411,417818365.0404715,599176692.2172579,271010923.09797156,374372770.02204,394579148.9187016,237869662.99171597,606701783.7713683,148560675.25142795,422735443.6537102,...,572151910.6403494,439901099.2879787,304919206.69614714,721050657.3552696,132.102092,26.949226,none,NaN,NaN,NaN
1,166.0862755041856_0.5858793269230769,363477950.08953905,651303189.4392803,223933384.08641553,367612132.7783069,363360496.3342866,223205801.5668584,596553038.5969964,99965642.3202608,363523866.8105213,...,609614808.6123556,478803365.0144199,266983835.86724555,959568044.1300417,166.086276,35.152760,Massbank:PB006064 Phenylalanine|(2S)-2-amino-3...,NaN,NaN,NaN


In [64]:
masses = ['Theoretical m/z', '-1 H2O', '-2 H2O', '-3 H2O', 'Dimer', 'Trimer']
mztol = 10 #ppm
rttol = 9 #sec

In [65]:
feature['level_1_identification']=np.nan
feature['ppm_err']=np.nan
feature['rt_err']=np.nan

In [66]:
for idx in ids.index:
    tact = float(ids.loc[idx, 'RT sec'])
    
    for col in masses:
        mact = float(ids.loc[idx, col])
        for idx1 in feature.index:
            mobs = feature.loc[idx1, 'obs_mz']
            tobs = feature.loc[idx1, 'obs_rt']
            ppme = abs((mact-mobs)*1000000/float(mact))
            rte = abs((tact-tobs))
            
            #in case feature hasn't been assigned any lib match yet
            if ppme < mztol and rte < rttol:
                if pd.isnull(feature.loc[idx1, 'level_1_identification']): 
                    feature.loc[idx1, 'level_1_identification'] = ids.loc[idx, 'Name']
                    feature.loc[idx1, 'ppm_err']=ppme
                    feature.loc[idx1, 'rt_err']=rte
            #in case feature was assigned lib match but new match seems more accurate (lesser mz error & rt error < 0.15 m)
                elif ppme < feature.loc[idx1, "ppm_err"] and rte < rttol:
                    feature.loc[idx1, 'level_1_identification'] = ids.loc[idx,'Name']
                    feature.loc[idx1, 'ppm_err']=ppme
                    feature.loc[idx1, 'rt_err']=rte
            #in case feature was assigned lib match which is more accurate than the new match, make no change
                else: 
                    continue

In [68]:
# standard annotations found
feature['level_1_identification'].unique()

array([nan, 'Deoxycholic acid', 'b-Muricholic acid', 'Cholic acid',
       'a-Muricholic acid', 'Tauro-b-muricholic acid',
       'Chenodeoxycholic acid', 'Ursodeoxycholic acid', 'Taurocholic acid',
       'Lithocholic acid', 'Hyodeoxycholic acid', 'Taurodeoxycholic acid'], dtype=object)

In [69]:
# fill NANs woth "none"
feature['level_1_identification']=feature['level_1_identification'].fillna(value="none")

#delete 'ppm_err' and 'rt_err' columns for gnps annotations
feature.drop(['ppm_err', 'rt_err'], axis=1, inplace=True)

In [70]:
feature

#SampleID,#featureID,10422.17.F.10,10422.17.F.11,10422.17.F.12,10422.17.F.13,10422.17.F.3,10422.17.F.4,10422.17.F.5,10422.17.F.6,10422.17.F.7,...,10422.32.F.4,10422.32.F.5,10422.32.F.6,10422.32.F.7,10422.32.F.8,10422.32.F.9,obs_mz,obs_rt,level2_gnps,level_1_identification
0,132.1020923761221_0.4491537660256411,417818365.0404715,599176692.2172579,271010923.09797156,374372770.02204,394579148.9187016,237869662.99171597,606701783.7713683,148560675.25142795,422735443.6537102,...,278835766.3882692,591675934.303529,572151910.6403494,439901099.2879787,304919206.69614714,721050657.3552696,132.102092,26.949226,none,none
1,166.0862755041856_0.5858793269230769,363477950.08953905,651303189.4392803,223933384.08641553,367612132.7783069,363360496.3342866,223205801.5668584,596553038.5969964,99965642.3202608,363523866.8105213,...,276344621.4609494,601801823.8958603,609614808.6123556,478803365.0144199,266983835.86724555,959568044.1300417,166.086276,35.152760,Massbank:PB006064 Phenylalanine|(2S)-2-amino-3...,none
2,357.2784138555112_5.010000161030595,373010119.55232674,435299966.30379647,435849921.0834126,392079843.8921625,479809538.9549517,604022226.8888701,450688368.15204287,338018688.0039202,351623178.6585113,...,422084603.53361845,330066878.8931732,279712497.3909418,374946234.5400986,210245062.63768673,492174285.0353122,357.278414,300.600010,Spectral Match to 3.beta.-Hydroxy-5-cholenoic ...,Deoxycholic acid
3,104.1070008277893_0.3359217147435899,530218681.70972085,526654055.410282,220650036.66389745,490423509.762227,386665398.1254175,231921046.65578157,415931273.45851356,351438293.14226407,424729174.7062708,...,370002838.39459574,442259507.0676554,418912270.6658614,423527706.4934885,299363173.4537778,588373181.7669369,104.107001,20.155303,none,none
4,373.2735011621581_3.9791876602564105,350478282.39686227,710333455.3374097,585270016.8213968,600715632.301741,419050727.4609589,337573329.69389665,535634870.43608457,465673782.8194285,505418053.95629966,...,577675177.05133,526526953.8845931,632355790.5502381,707208052.0193648,304817225.4824474,734383697.6452729,373.273501,238.751260,Spectral Match to Cholic acid from NIST14,b-Muricholic acid
5,496.33975367154477_5.193326247987118,447061155.89179593,181916133.3388062,162988784.84624785,466151701.566379,891263216.3549309,552726195.4445323,231349171.646291,553002642.877882,118456351.87357737,...,586976201.3450794,163988094.02877572,263884505.6941217,556534464.0824897,158130589.76557833,576733212.505232,496.339754,311.599575,Spectral Match to Lyso-PC(16:0) from NIST14,none
6,118.08631741083585_0.3764403044871795,422961963.8465183,549539636.7641147,255817625.2793514,358719658.57339174,454623178.23659253,239932783.39001226,604796656.2014782,185140297.42450354,487363826.1766136,...,239579197.90515715,494408513.1159234,442650497.6968239,633303793.4955943,342925563.2268935,443172460.6608192,118.086317,22.586418,none,none
7,373.27348764725264_4.338406810897439,513391200.38879967,536791877.31420463,457220043.7653666,418305680.27995455,799390730.7553858,387211325.9015329,392476261.25450176,709478982.6054002,383569202.42665774,...,545710388.8198787,392736482.6735134,347674399.7469498,493572430.0138499,224453070.0529416,461842049.1848679,373.273488,260.304409,Spectral Match to Cholic acid from NIST14,Cholic acid
8,86.09642685376681_0.4473783653846155,389934380.6732999,612813994.7329236,252894181.59312475,386472978.312429,402519056.0386717,244410857.20752603,645886088.2716941,149984827.79461783,444038680.55499,...,294083723.1864178,646059664.5722501,648696941.5305847,497068615.9262684,325276189.2524485,737477243.8912868,86.096427,26.842702,none,none
9,391.2841105382007_4.130280368589744,464497143.4832501,605694052.5767651,486168472.0114287,446345207.585744,500337692.3325029,494462117.6017439,407986139.3980972,509927690.6570184,422585651.7056464,...,450291858.6891234,329257577.65083766,245449517.6860748,395182753.94650143,154698905.26289055,273086130.0337165,391.284111,247.816822,Spectral Match to 12-Ketodeoxycho

In [71]:
feature.to_csv('./data/haddad_6weeks_allFeatures_pqn_matched_wIDs.txt', sep = '\t', index = False)